In [1]:
import numpy as np
from pathlib import Path

from pymor.basic import *
from pymor.core.pickle import load

from RBInvParam.problems.problems import build_InstationaryModelIP

set_log_levels({
    'pymor' : 'WARN'
})

set_defaults({})


In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "cm",
    "font.size": 10,
    'text.latex.preamble': r'\usepackage{amsfonts} \usepackage{accents} \usepackage{mathrsfs}',
    'figure.dpi': 400
})

In [14]:
#WORK_DIR = Path('/run/media/benedikt/38E84473425D444F/Dokumente/Studium/Mathematik/PhD/parabolische_inverse_probleme')
WORK_DIR = Path('/home/benedikt/Dokumente/parabolische_inverse_probleme')
FOM_data_dir_path = WORK_DIR / Path('experiments/enforce_adm_region_corrected_suff_cond')
TR_data_dir_path = WORK_DIR / Path('experiments/test_enrichment_strategy_new')

FOM_data_path = FOM_data_dir_path / 'diffusion_FOM_300_time_dep'
TR_data_path = TR_data_dir_path / 'diffusion_TR_time_dep_1e-12'

setup_path = FOM_data_path / 'setup.pkl'
optimizer_parameter_path = FOM_data_path / 'optimizer_parameter.pkl'


with open(setup_path, 'rb') as file:
    setup = load(file)

with open(optimizer_parameter_path, 'rb') as file:
    optimizer_parameter = load(file)

if not 'FOM' in locals():
    FOM, grid_data, assembled_products = build_InstationaryModelIP(setup=setup)

    

data = {}

if FOM_data_path:
    with open(FOM_data_path / 'FOM_IRGNM_final.pkl', 'rb') as file:
        FOM_data = load(file)
    data['FOM'] = FOM_data

if TR_data_path:
    with open(TR_data_path / 'TR_IRGNM_final.pkl', 'rb') as file:
        TR_data = load(file)
    data['TR'] = TR_data
    


In [11]:
from RBInvParam.products import BochnerProductOperator

q_FOM = data['FOM']['q'][-1]
q_TR = data['TR']['q'][-1]

nt = setup['dims']['nt']
#nt = 1

bochner_L2 = BochnerProductOperator(
    product=NumpyMatrixOperator(
        matrix = assembled_products['l2'],
        source_id = FOM.Q.id,
        range_id = FOM.Q.id
    ),
    delta_t=setup['model_parameter']['delta_t'],
    space = FOM.Q,
    nt = nt
)

print("L2 rel error:")
result = np.sqrt(bochner_L2.apply2(q_FOM-q_TR, q_FOM-q_TR)[0,0]) / np.sqrt(bochner_L2.apply2(q_FOM, q_FOM)[0,0]) * 100
print(f"{result:.2f}")

def L2error(q_FOM, q_TR) -> float:
    return np.sqrt(bochner_L2.apply2(q_FOM-q_TR, q_FOM-q_TR)[0,0]) / np.sqrt(bochner_L2.apply2(q_FOM, q_FOM)[0,0])

##############################################################################################

bochner_H1 = BochnerProductOperator(
    product=NumpyMatrixOperator(
        matrix = assembled_products['h1'],
        source_id = FOM.Q.id,
        range_id = FOM.Q.id
    ),
    delta_t=setup['model_parameter']['delta_t'],
    space = FOM.Q,
    nt = nt
)

print("H1 rel error:")
result = np.sqrt(bochner_H1.apply2(q_FOM-q_TR, q_FOM-q_TR)[0,0]) / np.sqrt(bochner_H1.apply2(q_FOM, q_FOM)[0,0]) * 100
print(f"{result:.2f}")

def H1error(q_FOM, q_TR) -> float:
    return np.sqrt(bochner_H1.apply2(q_FOM-q_TR, q_FOM-q_TR)[0,0]) / np.sqrt(bochner_H1.apply2(q_FOM, q_FOM)[0,0])
    


L2 rel error:
57.01
H1 rel error:
116.21


In [10]:
# import pandas as pd

# columns = ['Algorithm', 'time [s]', 'speed up', 'FOM solves', r'$n_Q$', r'$n_V$', 'o. iter']

# rows = []
# algo = 'FOM'
# FOM_solves = data[algo]['FOM_num_calls']['solve_state'] + data[algo]['FOM_num_calls']['solve_adjoint'] + \
#              data[algo]['FOM_num_calls']['solve_linearized_state'] + data[algo]['FOM_num_calls']['solve_linearized_adjoint']
# row = [algo,
#        int(data[algo]['total_runtime'][-1]),
#        '--',
#        FOM_solves,
#        '--',
#        '--',
#        len(data[algo]['J'])-1]
# rows.append(row)

# algo = 'TR'
# FOM_solves = data[algo]['FOM_num_calls']['solve_state'] + data[algo]['FOM_num_calls']['solve_adjoint'] + \
#              data[algo]['FOM_num_calls']['solve_linearized_state'] + data[algo]['FOM_num_calls']['solve_linearized_adjoint']

# row = [algo,       
#        int(data[algo]['total_runtime'][-1]),
#        '--',
#        FOM_solves,
#        data[algo]['dim_Q_r'][-1],
#        data[algo]['dim_V_r'][-1],
#        len(data[algo]['J'])-1]
# rows.append(row)


# df = pd.DataFrame.from_records(rows, columns=columns)
# df['speed up'][1] = df['time [s]'][0] / df['time [s]'][1]

ValueError: 7 columns passed, passed data had 11 columns

In [15]:
import pandas as pd

columns = ['Algorithm', 
           r'$\epsilon_{\text{POD,Q}}$', 
           r'$\epsilon_{\text{POD,V}}$', 
           r'$L^2$-rel. err.',
           r'$H^1$-rel. err.',
           'time [s]', 
           'speed up', 
           'FOM solves', 
           r'$n_Q$', 
           r'$n_V$', 
           'o. iter']

rows = []
algo = 'FOM'
FOM_solves = data[algo]['FOM_num_calls']['solve_state'] + data[algo]['FOM_num_calls']['solve_adjoint'] + \
             data[algo]['FOM_num_calls']['solve_linearized_state'] + data[algo]['FOM_num_calls']['solve_linearized_adjoint']
row = [algo,
       '--',
       '--',
       '--',
       '--',
       int(data[algo]['total_runtime'][-1]),
       '--',
       FOM_solves,
       '--',
       '--',
       len(data[algo]['J'])-1]
rows.append(row)

FOM_total_time = int(data[algo]['total_runtime'][-1])

TR_data_paths = [
    #'diffusion_TR_time_dep_1e-9',
    'diffusion_TR_time_dep_1e-9_1e-6',
    'diffusion_TR_time_dep_1e-12',
    'diffusion_TR_time_dep_1e-16',
    'diffusion_TR_time_dep_1e-16_1e-6',
    'diffusion_TR_time_dep_1e-16_1e-9',
]

for TR_data_path in TR_data_paths:
    with open(TR_data_dir_path / TR_data_path / 'TR_IRGNM_final.pkl', 'rb') as file:
        TR_data = load(file)
    data['TR'] = TR_data

    algo = 'TR'
    FOM_solves = data[algo]['FOM_num_calls']['solve_state'] + data[algo]['FOM_num_calls']['solve_adjoint'] + \
                 data[algo]['FOM_num_calls']['solve_linearized_state'] + data[algo]['FOM_num_calls']['solve_linearized_adjoint']
    
    row = [algo,
           f"{data['TR']['optimizer_parameter']['enrichment']['parameter_HaPOD_tol']:.2e}",
           f"{data['TR']['optimizer_parameter']['enrichment']['state_HaPOD_tol']:.2e}",
           f"{L2error(data['FOM']['q'][-1],data['TR']['q'][-1]):.2e}",
           f"{H1error(data['FOM']['q'][-1],data['TR']['q'][-1]):.2e}",
           int(data[algo]['total_runtime'][-1]),
           FOM_total_time / int(data[algo]['total_runtime'][-1]),
           FOM_solves,
           data[algo]['dim_Q_r'][-1],
           data[algo]['dim_V_r'][-1],
           len(data[algo]['J'])-1]
    rows.append(row)
    
    
df = pd.DataFrame.from_records(rows, columns=columns)
    #df['speed up'][1] = df['time [s]'][0] / df['time [s]'][1]

In [16]:
print(df.to_latex(index=False, float_format="{:.2f}".format, column_format='l|cccccc'))

\begin{tabular}{l|cccccc}
\toprule
Algorithm & $\epsilon_{\text{POD,Q}}$ & $\epsilon_{\text{POD,V}}$ & $L^2$-rel. err. & $H^1$-rel. err. & time [s] & speed up & FOM solves & $n_Q$ & $n_V$ & o. iter \\
\midrule
FOM & -- & -- & -- & -- & 85436 & -- & 47099 & -- & -- & 23 \\
TR & 1.00e-09 & 1.00e-06 & 1.58e-02 & 1.33e-01 & 10840 & 7.88 & 32 & 28 & 179 & 15 \\
TR & 1.00e-12 & 1.00e-12 & 4.42e-03 & 4.55e-02 & 5368 & 15.92 & 16 & 81 & 281 & 7 \\
TR & 1.00e-16 & 1.00e-16 & 2.51e-03 & 1.75e-02 & 5338 & 16.01 & 14 & 224 & 234 & 6 \\
TR & 1.00e-16 & 1.00e-06 & 4.54e-03 & 4.21e-02 & 6266 & 13.63 & 20 & 368 & 100 & 9 \\
TR & 1.00e-16 & 1.00e-09 & 1.54e-03 & 1.43e-02 & 5354 & 15.96 & 14 & 222 & 212 & 6 \\
\bottomrule
\end{tabular}

